In [3]:
import numpy as np
import matplotlib.pyplot as plt
import uproot
import awkward as ak
import os
import scipy.optimize as sc
import matplotlib as mpl
import glob
from matplotlib.path import Path
from collections import Counter
from tqdm import tqdm

In [4]:
datapath = "/home/haue/repositories/Masters/data/" 


alpha_si_files = glob.glob(os.path.join(datapath+"singleprotons/Si", "*mlio.root"))
pfit_si_files = glob.glob(os.path.join(datapath+"singleprotons/Si/newpad1", "*mlio.root"))
narrow_pfit_si_files = glob.glob(os.path.join(datapath+"singleprotons/Si/newpad2", "*mlio.root"))


In [5]:
# Storing the data in a dictionary for easy access

### Should be a more intuitive version than the ones used in Bananachecks...

data_files = {
    "Si_a": alpha_si_files,
    "Si_p": pfit_si_files,
    "Si_np": narrow_pfit_si_files

}
data_collection = {
    "Si_a": {},
    "Si_p": {},
    "Si_np": {}
}

data_collection = {isotope: {f"id{i}": {key: [] for key in ["E", "fEdep", "bEdep", "angle", "Edep", "FI", "BI"]} for i in range(4)} for isotope in data_files}
data_collection["All"] = {f"id{i}": {key: [] for key in ["E", "fEdep", "bEdep", "angle", "Edep", "FI", "BI"]} for i in range(4)}


for isotope, files in data_files.items():
    #file_progress = tqdm(files, desc=f"Processing {isotope}", unit="file", total=len(files), dynamic_ncols=True)
    file_progress = tqdm(files, desc=f"Processing {isotope}", unit="file", total=len(files), bar_format="{l_bar}{bar} {n_fmt}/{total_fmt} [{elapsed}<{remaining}, {rate_fmt}]")

    for file in file_progress:
        
        ## accesses the tree "a" in the root file
        r = uproot.open(file)["a"]
        ##Dummy array
        dum = r.arrays(["id", "E", "fEdep", "bEdep", "angle", "Edep", "FI", "BI"])
        
        ## converts it to an awkward array
        ids = ak.flatten(dum["id"])
        Es = ak.nan_to_num(ak.flatten(dum["E"]) / 1e3)
        fEdeps = ak.nan_to_num(ak.flatten(dum["fEdep"]) / 1e3)
        bEdeps = ak.nan_to_num(ak.flatten(dum["bEdep"]) / 1e3)
        angles = ak.nan_to_num(np.rad2deg(ak.flatten(dum["angle"])))
        Edeps = ak.nan_to_num(ak.flatten(dum["Edep"]) / 1e3)
        FIs = ak.flatten(dum["FI"])
        BIs = ak.flatten(dum["BI"])
        
        ##Creates entries in data_collection for each key and for each detector for each key
        for i in range(4):
            mask = (ids==i)
            detector_name = f"id{i}"
            data_collection[isotope][detector_name]["E"].extend(Es[mask])
            data_collection[isotope][detector_name]["fEdep"].extend(fEdeps[mask])
            data_collection[isotope][detector_name]["bEdep"].extend(bEdeps[mask])
            data_collection[isotope][detector_name]["angle"].extend(angles[mask])
            data_collection[isotope][detector_name]["Edep"].extend(Edeps[mask])
            data_collection[isotope][detector_name]["FI"].extend(FIs[mask])
            data_collection[isotope][detector_name]["BI"].extend(BIs[mask])
        file_progress.set_postfix({"Completed": f"{file_progress.n}/{file_progress.total}"})
    #print(f"Processing done for {isotope}!")
    

tqdm.write("Combining all isotopes into 'All'...")      
## Create the "All" branch in data_collection
for i in range(4):
    detector_key = f"id{i}"
    for key in ["E", "fEdep", "bEdep", "angle", "Edep", "FI", "BI"]:
        data_collection["All"][detector_key][key] = sum((data_collection[isotope][detector_key][key] for isotope in data_files), [])

tqdm.write("✅ All data combined successfully!")


Processing Si_a: 100%|██████████ 11/11 [00:52<00:00,  4.75s/file]
Processing Si_p: 100%|██████████ 11/11 [00:22<00:00,  2.03s/file]
Processing Si_np: 100%|██████████ 11/11 [00:37<00:00,  3.39s/file]


Combining all isotopes into 'All'...
✅ All data combined successfully!
